In [2]:
# Import dependencies
import pandas as pd

# Reading data in Pandas

### OMDb API data

In [5]:
# Reading json file from raw data to pandas
df_raw = pd.read_json('../Output/OMDb_Utelly/OMDb_250.json')
# df_raw.head()

# Cleaning data by removing rows with no data available
omdb_df = df_raw.loc[df_raw['Error'].isna()]

omdb_df.head(2)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error
0,The Shawshank Redemption,1994.0,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton",A banker convicted of uxoricide forms a friend...,...,9.3,"2,957,050",tt0111161,movie,N/A,"$28,767,189",N/A,N/A,True,NaN
1,The Godfather,1972.0,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan",The aging patriarch of an organized crime dyna...,...,9.2,"2,057,473",tt0068646,movie,N/A,"$136,381,073",N/A,N/A,True,NaN


### Utelly API data

In [6]:
# Reading csv file from raw data to pandas
df_raw_service = pd.read_csv('../Output/OMDb_Utelly//streaming_df.csv')
df_raw_service.head(2)

,IMDb ID,Title,Streaming Service,Streaming URL
0,tt0111161,The Shawshank Redemption,Amazon Instant Video,https://www.amazon.com/gp/video/detail/B001EBV...
1,tt0111161,The Shawshank Redemption,Google Play,https://play.google.com/store/movies/details/T...


In [7]:
# Cleaning data by removing rows with no data available
df_raw_service = df_raw_service.dropna(how = 'any')

### Rapid API data

In [8]:
# Reading csv file from raw data to pandas
df_raw_netflix = pd.read_csv('../Output/uNoGS/netflix_all_pages.csv')
df_raw_netflix.head(2)

,Unnamed: 0,netflixid,title,image,synopsis,rating,type,released,runtime,largeimage,unogsdate,imdbid,download
0,0,81218363,Sillu Karuppatti,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,From first crushes to post-marriage relationsh...,8.8,movie,2019,2h13m,https://occ-0-37-33.1.nflxso.net/dnm/api/v6/ev...,2020-01-26,tt9614988,0.0
1,1,70272846,30 for 30: Survive and Advance,http://art-0.nflximg.net/40031/f380436ffeb09a7...,"In 1983, the NC State Wolfpack, coached by Jim...",8.7,movie,2013,1h41m,http://cdn0.nflximg.net/images/8232/11928232.jpg,2015-04-14,tt2751904,0.0


In [9]:
# Cleaning data by removing rows with no data available
df_raw_netflix = df_raw_netflix.dropna(how = 'any')

### Google Webscraping data

In [15]:
# Reading csv file from raw data to pandas
df_raw_google = pd.read_csv('../Output/IMDb/Google_Scraping_Cleaned_Complete.csv')
df_raw_google.head(2)

,Title,Streaming On,Price
0,The Shawshank Redemption,Apple TV,"25.000,00"
1,The Godfather,Netflix,Langganan


In [16]:
# Cleaning data by removing rows with no data available
df_raw_google = df_raw_google.dropna(how = 'any')

# Transform

### Creating top_imdb  - 1NF

In [20]:
# Show IMDB ID, Title
df_title = omdb_df[['imdbID', 'Title']].copy()

# Renaming column headers
df_title = df_title.rename(columns = {'imdbID': 'imdb_id',
                                     'Title': 'title'})
df_title

,imdb_id,title
0,tt0111161,The Shawshank Redemption
1,tt0068646,The Godfather
2,tt0468569,The Dark Knight
3,tt0071562,The Godfather Part II
4,tt0050083,12 Angry Men
...,...,...
244,tt0245712,Amores Perros
245,tt0032976,Rebecca
246,tt1454029,The Help
247,tt0061512,Cool Hand Luke


In [21]:
# Saving DataFrame to csv file
df_title.to_csv('../Output/Transform_Results/top_imdb.csv', index=False)

### Creating movie - 1NF

In [23]:
# Show IMDB ID, Title, release year, runtime, movie rating, imdb rating, production
df_movie = omdb_df[['imdbID', 'Title', 'Year', 'Runtime', 'Rated', 'imdbRating', 'Production']].copy()

# Setting index as imdb_rank column
df_movie.reset_index(level=0, inplace=True)

# Tranforming float number from year.0 to year
year_df = df_movie['Year'].astype(float).map("{:.0f}".format)
year_df = pd.DataFrame(year_df)

# Tranforming runtime column by splitting string 
runtime_df = df_movie['Runtime'].str.split(" ", expand=True)

# Naming columns
runtime_df.columns = ['number', 'string']

# Dropping minutes
runtime_df = runtime_df.drop(columns = 'string')

# Renaming column headers
df_movie = df_movie.rename(columns = {'index': 'imdb_rank',
                                     'imdbID': 'imdb_id',
                                     'Title': 'title',
                                     'Year': 'year-old',
                                     'Runtime': 'runtime-old',
                                     'Rated': 'rated',
                                     'imdbRating': 'imdb_rating',
                                     'Production': 'production'
                                    })

# Combining dataframes by assign function
df_combined_movie = df_movie.assign(year = year_df)
df_combined_movie = df_combined_movie.assign(runtime = runtime_df)

# Show IMDB ID, Title, release year, runtime, movie rating, imdb rating, production
df_combined_movie = df_combined_movie [['imdb_id','imdb_rank', 'title', 'year', 'runtime', 'rated', 'imdb_rating', 'production']]

# Cleaning data by removing rows with no data available
df_combined_movie = df_combined_movie.dropna(how = 'any')
df_combined_movie

,imdb_id,imdb_rank,title,year,runtime,rated,imdb_rating,production
0,tt0111161,0,The Shawshank Redemption,1994,142,R,9.3,N/A
1,tt0068646,1,The Godfather,1972,175,R,9.2,N/A
2,tt0468569,2,The Dark Knight,2008,152,PG-13,9.0,N/A
3,tt0071562,3,The Godfather Part II,1974,202,R,9.0,N/A
4,tt0050083,4,12 Angry Men,1957,96,Approved,9.0,N/A
...,...,...,...,...,...,...,...,...
243,tt0245712,244,Amores Perros,2000,154,R,8.0,N/A
244,tt0032976,245,Rebecca,1940,130,Approved,8.1,N/A
245,tt1454029,246,The Help,2011,146,PG-13,8.1,N/A
246,tt0061512,247,Cool Hand Luke,1967,127,Approved,8.1,N/A


In [24]:
# Saving DataFrame to csv file
df_combined_movie.to_csv('../Output/Transform_Results/movie.csv', index=False)

### Creating movie_actor  - 1NF

In [25]:
# Show IMDB ID, Actors
df_actor = omdb_df[['imdbID', 'Actors']].copy()

# Dropping an N/A values
df_actor = df_actor.dropna(how = 'any')

# Creating an actor list to set up a conversion into a DataFrame
movie_actor_list = []

# Iterate through actors dataframe to clean up data into a dictionary
for index, row in df_actor.iterrows():

    all_actors = df_actor.loc[index,'Actors']
    actors_list = all_actors.split(', ')

    for actor in actors_list:
        
        movie_actor_dict = {}
        
        movie_actor_dict['imdb_id'] = df_actor.loc[index,'imdbID']
        movie_actor_dict['actor'] = actor
        
        movie_actor_list.append(movie_actor_dict)

# Convert actor list into a Dataframe
movie_actor_df = pd.DataFrame(movie_actor_list)

# Cleaning data by removing rows with no data available
movie_actor_df = movie_actor_df.dropna(how = 'any')
movie_actor_df

,imdb_id,actor
0,tt0111161,Tim Robbins
1,tt0111161,Morgan Freeman
2,tt0111161,Bob Gunton
3,tt0068646,Marlon Brando
4,tt0068646,Al Pacino
...,...,...
739,tt0061512,George Kennedy
740,tt0061512,Strother Martin
741,tt5323662,Miyu Irino
742,tt5323662,Saori Hayami


In [26]:
# Saving DataFrame to csv file
movie_actor_df.to_csv('../Output/Transform_Results/movie_actor.csv', index=False)

### Creating movie_director  - 1NF

In [27]:
# Show IMDB ID, Directors
df_director = omdb_df[['imdbID', 'Director']].copy()

# Dropping any N/A values
df_director = df_director.dropna(how = 'any')

# Creating a director list to set up a conversion into a DataFrame
movie_director_list = []

# Iterate through directors dataframe to clean up data into a dictionary
for index, row in df_director.iterrows():

    all_directors = df_director.loc[index,'Director']
    directors_list = all_directors.split(', ')

    for director in directors_list:
        
        movie_director_dict = {}
        
        movie_director_dict['imdb_id'] = df_director.loc[index,'imdbID']
        movie_director_dict['director'] = director
        
        movie_director_list.append(movie_director_dict)

# Convert director list into a Dataframe
movie_director_df = pd.DataFrame(movie_director_list)

# Cleaning data by removing rows with no data available
movie_director_df = movie_director_df.dropna(how = 'any')
movie_director_df

,imdb_id,director
0,tt0111161,Frank Darabont
1,tt0068646,Francis Ford Coppola
2,tt0468569,Christopher Nolan
3,tt0071562,Francis Ford Coppola
4,tt0050083,Sidney Lumet
...,...,...
273,tt0032976,Alfred Hitchcock
274,tt1454029,Tate Taylor
275,tt0061512,Stuart Rosenberg
276,tt5323662,Taichi Ishidate


In [28]:
# Saving DataFrame to csv file
movie_director_df.to_csv('../Output/Transform_Results/movie_director.csv', index=False)

### Creating Streaming Service Utelly  - 1NF

In [102]:
# Show Streaming service
df_service = df_raw_service['Streaming Service']
df_service = pd.DataFrame(df_service)

# Renaming column header
df_service = df_service.rename(columns= {'Streaming Service': 'service_name'})

# To see unique values in Streaming Service column of df_raw_service
df_service.service_name.unique()

array(['Amazon Instant Video', 'Google Play', 'iTunes', 'Disney+',
       'Amazon Prime Video', 'Netflix'], dtype=object)

In [105]:
# Group Streaming services
grp_service = df_service.groupby('service_name')
df_service = pd.DataFrame(grp_service)

# Renaming columns
df_service.columns = ['service_name', 'info']

# Declaring a list for unqiue service id column
service_id = ['ss_1', 'ss_2', 'ss_3', 'ss_4', 'ss_5',
             'ss_6']

# Creating a service id column and adding the service_id list
df_service['service_id']= service_id

# Show service_id and service_name
df_service = df_service[['service_id', 'service_name']]

# Cleaning data by removing rows with no data available
df_service = df_service.dropna(how = 'any')
df_service

,service_id,service_name
0,ss_1,Amazon Instant Video
1,ss_2,Amazon Prime Video
2,ss_3,Disney+
3,ss_4,Google Play
4,ss_5,Netflix
5,ss_6,iTunes


In [106]:
# Saving DataFrame to csv file
df_service.to_csv('../Output/Transform_Results/streaming_service.csv', index=False)

### Creating Streaming Service Google  - 1NF

In [120]:
# Show Streaming service
df_service_google = df_raw_google['Streaming On']
df_service_google = pd.DataFrame(df_service_google)

# Renaming column header
df_service_google = df_service_google.rename(columns= {'Streaming On': 'service_name'})

# To see unique values in Streaming Service column of df_raw_service
df_service_google.service_name.unique()


array(['Apple TV', 'Netflix', 'Vidio', 'Google Play Film & TV',
       'Amazon Prime Video', 'Not Available', 'iQIYI'], dtype=object)

In [122]:
df_service_google = df_service_google[df_service_google.service_name != 'Not Available']

In [123]:
df_service_google.service_name.unique()

array(['Apple TV', 'Netflix', 'Vidio', 'Google Play Film & TV',
       'Amazon Prime Video', 'iQIYI'], dtype=object)

In [125]:
df_service_google

,service_name
0,Apple TV
1,Netflix
2,Vidio
3,Google Play Film & TV
4,Apple TV
...,...
324,Google Play Film & TV
326,Netflix
328,Apple TV
329,Google Play Film & TV


In [126]:
# Group Streaming services
grp_service = df_service_google.groupby('service_name')
df_service_google= pd.DataFrame(grp_service)
df_service_google

# Renaming columns
df_service_google.columns = ['service_name', 'info']

# Dropping info column
df_service_google = df_service_google.drop(columns='info')

df_service_google 

# Declaring a list for unqiue service id column
service_google_id = ['gg_1', 'gg_2', 'gg_3', 'gg_4', 'gg_5',
             'gg_6']

# Creating a service id column and adding the service_id list
df_service_google['service_id']= service_google_id

# Show service_id and service_name
df_service_google = df_service_google[['service_id', 'service_name']]
df_service_google

,service_id,service_name
0,gg_1,Amazon Prime Video
1,gg_2,Apple TV
2,gg_3,Google Play Film & TV
3,gg_4,Netflix
4,gg_5,Vidio
5,gg_6,iQIYI


In [127]:
# Saving DataFrame to csv file
df_service_google.to_csv('../Output/Transform_Results//google_streaming_service.csv', index=False)

### Creating streaming_service_utelly - 1NF

In [142]:
# Show IMDB id, Title, Streaming service
df_streaming = df_raw_service[['IMDb ID', 'Title', 'Streaming Service']].copy()

# Renaming column headers
df_streaming = df_streaming.rename(columns={
    'IMDb ID': 'imdb_id',
    'Title': 'title',
    'Streaming Service': 'service_name'
})

# Instead of assign, keep the original service_name column
df_combined_streaming = df_streaming.copy()

# Create service ID mapping dictionary
service_id_mapping = {
    'Amazon Instant Video': 'ss_1',
    'Amazon Prime Video': 'ss_2',
    'Disney+': 'ss_3',
    'Google Play': 'ss_4',
    'Netflix': 'ss_5',
    'iTunes': 'ss_6'
}

# Map service names to IDs using map function
df_combined_streaming['service_id'] = df_combined_streaming['service_name'].map(service_id_mapping)

# Show service id and IMDB id
df_streaming_id = df_combined_streaming[['imdb_id', 'service_id']]

# Cleaning data by removing rows with no data available
df_service_utelly = df_streaming_id.dropna(how='any')

# Display result
print(f"Original rows: {len(df_streaming_id)}")
print(f"Rows after cleaning: {len(df_service_utelly)}")
df_service_utelly

Original rows: 761
Rows after cleaning: 761


,imdb_id,service_id
0,tt0111161,ss_1
1,tt0111161,ss_4
2,tt0111161,ss_6
3,tt0068646,ss_1
4,tt0068646,ss_4
...,...,...
758,tt0061512,ss_6
759,tt0061512,ss_2
760,tt5323662,ss_1
761,tt5323662,ss_4


In [143]:
# Saving DataFrame to csv file
df_streaming_id.to_csv('../Output/Transform_Results//utelly_movie_streaming.csv', index=False)

### Creating streaming_service_google - 1NF

In [154]:
# Show Title, Streaming On, Price
df_service_google = df_raw_google[['Title', 'Streaming On', 'Price' ]].copy()

# Renaming column headers
df_service_google= df_service_google.rename(columns = {'Title': 'title',
                                     'Streaming On': 'service_name',
                                     'Price': 'price'
                                    })

# To see unique values in Streaming Service column of df_raw_service
df_service_google.service_name.unique()

array(['Apple TV', 'Netflix', 'Vidio', 'Google Play Film & TV',
       'Amazon Prime Video', 'Not Available', 'iQIYI'], dtype=object)

In [155]:
df_service_google = df_service_google[df_service_google.service_name != 'Not Available']

In [156]:
df_service_google.service_name.unique()

array(['Apple TV', 'Netflix', 'Vidio', 'Google Play Film & TV',
       'Amazon Prime Video', 'iQIYI'], dtype=object)

In [157]:
# Instead of assign, keep the original columns
df_combined_google = df_service_google.copy()

# Create service ID mapping dictionary
google_service_mapping = {
    'Amazon Prime Video': 'gg_1',
    'Apple TV': 'gg_2',
    'Google Play Film & TV': 'gg_3',
    'Netflix': 'gg_4',
    'Vidio': 'gg_5',
    'iQIYI': 'gg_6'
}

# Map service names to IDs using map function
df_combined_google['google_service_id'] = df_combined_google['service_name'].map(google_service_mapping)

# Select and reorder columns
df_service_google = df_combined_google[['title', 'google_service_id', 'price']]

# Cleaning data by removing rows with no data available
df_service_google = df_service_google.dropna(how='any')

# Display result
print(f"Original rows: {len(df_combined_google)}")
print(f"Rows after cleaning: {len(df_service_google)}")
df_service_google

Original rows: 208
Rows after cleaning: 208


,title,google_service_id,price
0,The Shawshank Redemption,gg_2,"25.000,00"
1,The Godfather,gg_4,Langganan
2,The Godfather,gg_5,Langganan
3,The Godfather,gg_3,Langganan
4,The Godfather,gg_2,Langganan
...,...,...,...
324,Groundhog Day,gg_3,"25.000,00"
326,Rebecca,gg_4,Langganan
328,Cool Hand Luke,gg_2,"25.000,00"
329,Cool Hand Luke,gg_3,"25.000,00"


In [158]:
# Saving DataFrame to csv file
df_service_google.to_csv('../Output/Transform_Results//google_movie_streaming.csv', index=False)

### Creating netflix_movie - 1NF

In [160]:
# Show Netflix ID, IMDB ID, Title
df_netflix = df_raw_netflix[['netflixid', 'imdbid', 'title' ]].copy()

# Renaming column headers
df_netflix = df_netflix.rename(columns = {'netflixid': 'netflix_id',
                                     'imdbid': 'imdb_id',
                                    })

# Cleaning data by removing rows with no data available
df_netflix = df_netflix.dropna(how = 'any')
df_netflix

,netflix_id,imdb_id,title
0,81218363,tt9614988,Sillu Karuppatti
1,70272846,tt2751904,30 for 30: Survive and Advance
2,81086533,tt1606375,Downton Abbey
3,81281469,tt8784906,Ani... Dr. Kashinath Ghanekar
4,81327516,tt0073486,One
...,...,...,...
451,81505042,tt4178092,The Gift
452,81600613,tt0328349,Sleepless Nights
453,70018320,tt0213905,Crane World
454,81679591,tt6425816,One Piece Heart of Gold


In [161]:
# Saving DataFrame to csv file
df_netflix.to_csv('../Output/Transform_Results//netflix_movie.csv', index=False)